# Regular Expressions

## Announcements

## Final project discussion

## Review

## Looking Back

- SQL, Relational Databases
   - Selecting, sorting, limiting, joins
   - Using SQLite in Colab with `%%sql`

- Data manipulation with Pandas
   - DataFrames and Series's
   - Import/Exporting to SQL
   - Pulling tables from web
   - Selection, sorting, counting

- Split-Apply-Combine
   - Groupby in Pandas e.g. `.groupby('column').mean()`

- Visualization

- String pattern matching and extraction with regular expressions

- Semi-Structured data in MongoDB
    - JSON
    - selection, sorting
    - Aggregations
    - MapReduce concepts

## <center>Regular Expressions</center>
### <center>aka *regex*</center>

### Overview

Regular Expressions help you work with strings

*Pattern Matching*

e.g. Find all phone numbers on a web page

*Manipulation*

e.g. Match "{Lastname}, {Firstname}" in a set of records and rewrite it as "{Firstname} {Lastname}"

## Why?

- Checking whether an input is valid (i.e. password, phone number, email, etc.)
- Cleaning data
- More complex data subsetting
- Working with user inputs or other unstructured data

### Q: Where can you use regular expressions?

### A: Many, many places!

## In Python

In [2]:
import re
comment = "It was a dark and stormy night." 

Find a simple string:

In [3]:
re.findall('dark', comment)

['dark']

Find all sequences of one or more word characters:

In [4]:
re.findall('\w+', comment)

['It', 'was', 'a', 'dark', 'and', 'stormy', 'night']

## In SQL

SQLite doesn't support it, but most 'bigger' databases do. e.g.

**MySQL**

Select columns that match alphanumeric characters only:

```
SELECT * FROM table WHERE column REGEXP '^[A-Za-z0-9]+$';
```

**Postgresql**

Match strings that include foo, bar, or baz:

```
SELECT * FROM table WHERE value ~ 'foo|bar|baz';
```

## In Pandas

In [5]:
import pandas as pd
movies = pd.read_csv('https://raw.githubusercontent.com/organisciak/Scripting-Course/master/data/movielens_small.csv')
movies.sample()

,userId,rating,title,genres,timestamp,year
70502,318,4.0,Tin Cup,Comedy,862902803,1996


Find movies where there is a digit (`\d`) right before the end of the string (`$`):

In [6]:
matches = movies['title'].str.contains('\d$')
movies[matches].sample(10)

,userId,rating,title,genres,timestamp,year
12634,577,3.5,Fahrenheit 9/11,Documentary,1111551742,2004
26664,493,4.0,Toy Story 2,Adventure,954684761,1999
2210,602,3.0,Apollo 13,Adventure,842355955,1995
49320,289,0.5,Scream 2,Comedy,1328935429,1997
36620,418,1.5,Kill Bill: Vol. 1,Action,1132696099,2003
36567,88,4.0,Kill Bill: Vol. 1,Action,1239756823,2003
91830,564,1.0,House Party 2,Comedy,974838601,1991
26678,585,4.0,Toy Story 2,Adventure,974607441,1999
2127,353,2.0,Apollo 13,Adventure,1112268690,1995
65864,402,3.0,2012,Action,1443394856,2009


Find movies where the substring ' Part ' exists:

In [7]:
matches = movies.title.str.contains(' Part ')
movies[matches].sample(10)

,userId,rating,title,genres,timestamp,year
41900,162,4.0,Father of the Bride Part II,Comedy,852791965,1995
25155,406,3.0,"Godfather: Part II, The",Crime,1447951463,1974
25999,404,4.0,"Godfather: Part III, The",Crime,1026927893,1990
41934,624,3.0,Father of the Bride Part II,Comedy,1019128415,1995
25127,232,5.0,"Godfather: Part II, The",Crime,955086697,1974
41903,212,2.5,Father of the Bride Part II,Comedy,1218955060,1995
50257,134,3.5,Back to the Future Part III,Adventure,1361245906,1990
90790,353,1.0,Friday the 13th Part 3: 3D,Horror,1113053076,1982
84658,554,3.0,Friday the 13th Part 2,Horror,1012753261,1981
85816,157,2.5,"Karate Kid, Part II, The",Action,1291598091,1986


Find movies that are named "The ... of ..."

In [8]:
matches = movies.title.str.contains('^The .+ of ')
movies[matches].sample(10)

,userId,rating,title,genres,timestamp,year
36099,212,4.0,The Count of Monte Cristo,Action,1228789284,2002
68253,324,3.5,The Theory of Everything,Drama,1451526024,2014
97198,402,4.0,The Best of Me,Drama,1462948242,2014
75357,23,3.0,The Importance of Being Earnest,Comedy,1148669643,1952
68479,15,3.0,The Age of Adaline,Drama,1458506026,2015
36107,431,3.5,The Count of Monte Cristo,Action,1165548739,2002
96816,630,4.5,The Jinx: The Life and Deaths of Robert Durst,Documentary,1443807954,2015
97199,402,4.5,The Book of Life,Adventure,1443394238,2014
68255,371,5.0,The Theory of Everything,Drama,1462737932,2014
68251,149,4.0,The Theory of Everything,Drama,1437091480,2014


## In MongoDB

In [ ]:
from pymongo import MongoClient
client = MongoClient()
db = client.week7
collection = db.cooking

Find an recipe with an ingredient called "yellow ..."

In [10]:
collection.find_one({
    "ingredients": {"$regex": "yellow .*"}
})

{'_id': ObjectId('5cedb796db075a25e4ac71b5'),
 'cuisine': 'southern_us',
 'id': 25693,
 'ingredients': ['plain flour',
  'ground pepper',
  'salt',
  'tomatoes',
  'ground black pepper',
  'thyme',
  'eggs',
  'green tomatoes',
  'yellow corn meal',
  'milk',
  'vegetable oil']}

After unwinding the recipes to one doc per ingredient, find ingredients with a qualified salt:

In [24]:
pipeline = [
    { "$unwind": "$ingredients" },
    { "$project": {"ingredients": 1, "_id":0} },
    { "$match":{
        "ingredients": {"$regex": "^.+ salt" }
        }
    },
    { "$limit": 5 }
]
results = collection.aggregate(pipeline)
list(results)

[{'ingredients': 'sea salt'},
 {'ingredients': 'kosher salt'},
 {'ingredients': 'fine sea salt'},
 {'ingredients': 'kosher salt'},
 {'ingredients': 'kosher salt'}]

Count the qualified salt types:

In [25]:
pipeline = [
    { "$unwind": "$ingredients" },
    { "$project": {"ingredients": 1, "_id":0} },
    { "$match":{ "ingredients": {"$regex": "^.+ salt$" } } },
    { "$group":{
        "_id": "$ingredients", "count": {"$sum": 1} } 
    },
    { "$sort": { "count": -1} },
    { "$limit": 20 }
]
results = collection.aggregate(pipeline)
list(results)

[{'_id': 'kosher salt', 'count': 3113},
 {'_id': 'sea salt', 'count': 940},
 {'_id': 'coarse salt', 'count': 578},
 {'_id': 'fine sea salt', 'count': 285},
 {'_id': 'garlic salt', 'count': 240},
 {'_id': 'seasoning salt', 'count': 131},
 {'_id': 'table salt', 'count': 79},
 {'_id': 'coarse sea salt', 'count': 68},
 {'_id': 'coarse kosher salt', 'count': 64},
 {'_id': 'celery salt', 'count': 52},
 {'_id': 'fine salt', 'count': 24},
 {'_id': 'onion salt', 'count': 15},
 {'_id': 'rock salt', 'count': 14},
 {'_id': 'black salt', 'count': 12},
 {'_id': 'pickling salt', 'count': 12},
 {'_id': 'Himalayan salt', 'count': 11},
 {'_id': 'celtic salt', 'count': 9},
 {'_id': 'maldon sea salt', 'count': 8},
 {'_id': 'smoked sea salt', 'count': 6},
 {'_id': 'iodized salt', 'count': 4}]

### Note on variation

- Regular Expressions are *close* to standard, but different implementations are slightly different.

## Basics of Regular Expressions

In this class: we'll cover the basics, practiced in Python and Pandas.

To follow along:

In [9]:
import re

In [10]:
text = "it was a dark and stormy night."
re.findall('\w+', text)

['it', 'was', 'a', 'dark', 'and', 'stormy', 'night']

## Wild Cards

`a` - Match the letter 'a'. Same for most other characters

In [11]:
text = "Colorado"
re.findall('o', text)

['o', 'o', 'o']

In [12]:
text = "Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo"
re.findall('Buffalo buffalo', text)

['Buffalo buffalo', 'Buffalo buffalo', 'Buffalo buffalo']

`.` - Match any single character

In [13]:
text = "who, what, where, why, and how"
re.findall('wh.', text)

['who', 'wha', 'whe', 'why']

In [14]:
text = "who, what, where, why, and how"
re.findall('wh.,', text)

['who,', 'why,']

- `\w` - Match any word character (letters, number... support for non-English characters varies)
- `\W` - Match any non-word characters

In [15]:
text = "Who, what, where, why, and how"
re.findall('\w\w\w,', text)

['Who,', 'hat,', 'ere,', 'why,']

In [16]:
text = "Who, what, where, why, and how"
re.findall('\w', text)

['W',
 'h',
 'o',
 'w',
 'h',
 'a',
 't',
 'w',
 'h',
 'e',
 'r',
 'e',
 'w',
 'h',
 'y',
 'a',
 'n',
 'd',
 'h',
 'o',
 'w']

`\d` - Match any digit

In [17]:
text = "Party like it's 1999"
re.findall('\d', text)

['1', '9', '9', '9']

In [18]:
text = "Party like it's 1999"
re.findall('\d\d\d\d', text)

['1999']

### *What if I want to match an actual backslash or period?*

This is a problem:

In [19]:
text = "Dr. Jones Drinks Too Much"
re.findall('Dr.', text)

['Dr.', 'Dri']

Precede the character with a backslash

E.g.

- `.` - Matches *any* character
- `\.` - Matches a literal period

In [20]:
re.findall('Dr\.', text)

['Dr.']

## Reference (so far)
- `a` - Match the letter 'a'. Same for most other characters
- `\w` - Match any word character (letters, number... support for non-English characters varies)
- `\W` - Match any non-word characters
- `\d` - Match any digit
- `.` - Matches *any* character
- `\.` - Matches a literal period

Let's try the first few lab questions - 1.1. to 1.4.

`\s` - Match any whitespace character (space, tabs, line breaks sometimes)

*What will this return?*

In [ ]:
text = "The quick brown fox jumped over the lazy yellow dog"
re.findall('\s....\s', text)

`[ab]` - Group of multiple possible characters - in this case 'a' or 'b'

In [21]:
text = "The quick brown fox jumped over the lazy yellow dog"
re.findall('[Tt]he', text)

['The', 'the']

- `[a-z]` matches any character from a to z
- `[A-Z]` matches any character from A to Z

In [16]:
text = "text 1-800-SPAM for more information"
re.findall('[A-Z][A-Z][A-Z][A-Z]', text)

['SPAM']

Those square brackets are same as before, so you can group A-Z with other matches.

e.g. Match capital letters, digits, or hyphens:

In [23]:
text = "text 1-800-SPAM for more information"
re.findall('[\d\-A-Z]+', text)

['1-800-SPAM']

*Note above that a hyphen is another special character, so matching for a literal `-` is done with `\-`.*

Returning to the earlier data.

In [17]:
titles = movies.title.drop_duplicates()

"The (single word) of ..."

In [18]:
matches = titles.str.contains('^The \w+ of ')
titles[matches].sample(10)

99821                       The Face of an Angel
99139                        The End of the Tour
75357            The Importance of Being Earnest
68247                   The Theory of Everything
94869             The Possession of Michael King
93216                    The Diary of Anne Frank
96669    The Disappearance of Eleanor Rigby: Her
36095                  The Count of Monte Cristo
94735                  The Plague of the Zombies
97199                           The Book of Life
Name: title, dtype: object

In [19]:
matches = titles.str.contains(':')
titles[matches].sample(10)

85974                          Carnosaur 3: Primal Species
44442                        Kids in the Hall: Brain Candy
97936                                 Imagine: John Lennon
87777                                Underworld: Evolution
48488                              Speed 2: Cruise Control
94916                               Madonna: Truth or Dare
85005               Children of the Corn IV: The Gathering
88472                  The Butterfly Effect 3: Revelations
85954    Teenage Mutant Ninja Turtles II: The Secret of...
73527                         History of the World: Part I
Name: title, dtype: object

In [25]:
matches = titles.str.contains("^\w+\-\w+$")
titles[matches]

259                  Ben-Hur
12000             Spider-Man
40269                  X-Men
55032                  U-571
58796             Scooby-Doo
61252              Fail-Safe
65729                G-Force
66092               Kick-Ass
68396                Ant-Man
69332            Re-Animator
69765    Slaughterhouse-Five
81831                  K-PAX
83228                 BURN-E
83394               Non-Stop
83908               Bio-Dome
89557            Topsy-Turvy
93256               Cry-Baby
94106              She-Devil
95079               Kon-Tiki
96155              De-Lovely
96602               Catch-22
96617                Ben-hur
96717               Semi-Pro
98638                  T-Men
99056     Shakespeare-Wallah
99971        Straight-Jacket
Name: title, dtype: object

## Reference (so far)
- `a` - Match the letter `a`. Same for most other characters
- `.` - Match any single character
- `\w` - Match any word character (letters, number... support for non-English characters varies)
- `\W` - Match any non-word characters
- `\d` - Match any digit
- `.` - Matches *any* character
- `\.` - Matches a literal period
- `\s` - Match any whitespace character (space, tabs, line breaks sometimes)
- `[ab]` - Group of multiple possible characters - in this case `a` or `b`
- `[a-z]` matches any character from a to z
- `[A-Z]` matches any character from A to Z
- `[A-Zab]` matches any character from A to Z (`A-Z`), *or* `a` *or* `b`

## Repetition

`?` - One or zero of the preceding match

In [20]:
text = "color colour"
re.findall('colou?r', text)

['color', 'colour']

- `+` - One or more of the preceding match
- `*` - Zero or more of the preceding match

In [22]:
text = "GOAL GOOOOOOOOOAAAAAAL GAL"
re.findall('GO+A+L', text)

['GOAL', 'GOOOOOOOOOAAAAAAL']

In [34]:
text = "GOAL"
re.findall('GO+A+L', text)

['GOAL']

`*` and `+` are *greedy* in Python. They will grab as much as possible. 

In [23]:
text = "<p>Something or other</p><p>Yet more junk.</p>" 
re.findall('<p>.*</p>', text)

['<p>Something or other</p><p>Yet more junk.</p>']

In [26]:
text = "foo1@gmail.com;b-a-r@gmail.com;baz@gmail.com" 
re.findall('\w.*@gmail.com', text)

['foo1@gmail.com;b-a-r@gmail.com;baz@gmail.com']

`*?` is the *lazy* alternative, it will grab as little as possible.

['foo1@gmail.com', 'b-a-r@gmail.com', 'baz@gmail.com']

In [123]:
re.findall('\w.*?@gmail.com', text)

['foo1@gmail.com', 'b-a-r@gmail.com', 'baz@gmail.com']

## Reference (so far)

**Matching characters**
- `a` - Match the letter `a`. Same for most other characters
- `.` - Match any single character
- `\w` - Match any word character (letters, number... support for non-English characters varies)
- `\W` - Match any non-word characters
- `\d` - Match any digit
- `.` - Matches *any* character
- `\.` - Matches a literal period
- `\s` - Match any whitespace character (space, tabs, line breaks sometimes)

**Multiple Matches**
- `[ab]` - Group of multiple possible characters - in this case `a` or `b`
- `[a-z]` matches any character from a to z
- `[A-Z]` matches any character from A to Z
- `[A-Zab]` matches any character from A to Z (`A-Z`), *or* `a` *or* `b`

**Repeating**

*'greedy' means that it captures as much as it can, 'lazy' means it captures as little as possible.*
- `?` - One or zero of the preceding match
- `+` - One or more of the preceding match (greedy)
- `*` - Zero or more of the preceding match (greedy)
- `*?`, `+?`  - Lazy versions of `*` and `+`

## Start and End of Line

`^` - Start of line

In [27]:
text = "The quick brown fox jumped over the lazy yellow dog"
re.findall('^The', text)

['The']

In [44]:
re.findall('^The', text)

['The']

In [109]:
re.findall('^.*fox', text)

['The quick brown fox']

`$` - End of line

In [104]:
text = "The quick brown fox jumped over the lazy yellow dog"
re.findall('.......$', text)

['low dog']

In [28]:
text = "The quick brown fox jumped over the lazy yellow dog"
re.findall("^.*$", text)

['The quick brown fox jumped over the lazy yellow dog']

## Reference

**Matching characters**
- `a` - Match the letter `a`. Same for most other characters
- `.` - Match any single character
- `\w` - Match any word character (letters, number... support for non-English characters varies)
- `\W` - Match any non-word characters
- `\d` - Match any digit
- `.` - Matches *any* character
- `\.` - Matches a literal period
- `\s` - Match any whitespace character (space, tabs, line breaks sometimes)

**Multiple Matches**
- `[ab]` - Group of multiple possible characters - in this case `a` or `b`
- `[a-z]` matches any character from a to z
- `[A-Z]` matches any character from A to Z
- `[A-Zab]` matches any character from A to Z (`A-Z`), *or* `a` *or* `b`

**Repeating**

*'greedy' means that it captures as much as it can, 'lazy' means it captures as little as possible.*
- `?` - One or zero of the preceding match
- `+` - One or more of the preceding match (greedy)
- `*` - Zero or more of the preceding match (greedy)
- `*?`, `+?`  - Lazy versions of `*` and `+`

**Position**
- `^` - Start of line
- `$` - End of line

# Additional tips

Choose a range for repetition with `{min,max}`. e.g.

In [57]:
text = "YOLO"
re.search('YOLO{1,3}$', text) 

<_sre.SRE_Match object; span=(0, 4), match='YOLO'>

In [58]:
text = "YOLOOO"
re.search('YOLO{1,3}$', text)

<_sre.SRE_Match object; span=(0, 6), match='YOLOOO'>

In [66]:
text = "YOLOOOOOO"
re.search('YOLO{1,3}$', text)

*Negation*
    
Use the caret in square brackets: `[^aeiou]` means *not* a, e, i, o, or u

*Groups*
    
Treat multiple characters together, like if they were a single character.

Use parentheses. e.g:

In [53]:
text = "banana"
re.findall('^ba(na)+$', text)

['na']

In [66]:
text = "lololololololololololol"
re.findall('^l(ol)+$', text)

['ololololololololololol']

Capturing groups:

In [61]:
text = "Ketchup Catsup"
re.findall('(Ketch|Cats)up', text)

['Ketch', 'Cats']